#  create dummy data and calculate clusters

In [344]:
num_nodes = 60
margin = 5 # time margin for arrival before appointment and stay beyond
working_hours = [8 * 60, 18 * 60] # 8am to 6pm
percentage_of_appointments = 0.3
span_cost_coefficient = 20000 # adjust
slack = 20000 # adjust
penalty_factor = 300000
lunch_start = 11 * 60  # 11 AM in minutes
lunch_end = 13 * 60  # 1 PM in minutes
lunch_duration = 30
min_work_days = 6 # minimum number of work days per location

In [345]:
# week (mon-sun) constraints
max_days_off = 2
days_off = {5,7} # default Sunday no work
no_overnight_stays = {2}
max_overnight_stays = 3

number_of_half_days = 2
half_days = {3,5,6} # define via working hours
# lunch break based on working hours
# wenn keine Arbeitszeiten angegeben werden, werden jene von 8-18 Uhr (default) angenommen

In [346]:
from src.routing import create_nodes_dataframe, custom_clustering, plot_refined_clusters, assign_weekdays_to_clusters, plot_ind_route, plot_all_cluster_routes, create_data_model, plot_all_nodes_with_angles
import pandas as pd
import numpy as np
import concurrent.futures
from datetime import datetime, timedelta, time
import random
from collections import Counter

from scipy.spatial import distance

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [347]:
def create_nodes_dataframe(num_nodes, home_node_id, min_work_days, days_off=days_off, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=30, frac_fixed_app=.1):
    """Create a DataFrame containing nodes' information, including their attributes and distances."""
    node_ids = np.arange(0, num_nodes)
    
    def generate_opening_hours(min_work_days):
        """Generate opening hours for a random subset of days."""
        schedule = {}
        # Randomly select a minimal number of working days within a given week
        days = sorted(random.sample(range(1, 8), min_work_days))
        for day in days:
            # Generate random start hours and end hours within constraints
            start_hour = 8 + random.randint(0, 1)
            end_hour = start_hour + random.randint(8, 9)
            start_minute = random.randint(0, 59)
            end_minute = random.randint(0, 59)

            # Randomly add lunch break
            if random.random() < 0.5:
                lunch_start = 11 + random.randint(0, 1)
                lunch_end = lunch_start + 1
                lunch_start_minute = random.randint(0, 59)
                lunch_end_minute = random.randint(0, 59)

                # Creating datetime.time objects for each period
                schedule[day] = [
                    [time(start_hour, start_minute), time(lunch_start, lunch_start_minute)],
                    [time(lunch_end, lunch_end_minute), time(end_hour, end_minute)]
                ]
            else:
                # Creating datetime.time objects for a non-interrupted period
                schedule[day] = [
                    [time(start_hour, start_minute), time(end_hour, end_minute)]
                ]
        return schedule
    
    def convert_to_time(hours_dict): # duplicate
        """Convert string-based opening hours to datetime.time objects."""
        return {k: (datetime.strptime(v[0], "%H:%M").time(), datetime.strptime(v[1], "%H:%M").time()) for k, v in hours_dict.items()}

    def add_fixed_appointments(nodes_df, frac_fixed_app):
        selected_indices = nodes_df.sample(frac=frac_fixed_app, replace=True).index
        for idx in selected_indices:
            # day can't be in days_off
            day = random.choice([day for day in nodes_df.at[idx, 'opening_hours'].keys() if day not in days_off])
            start, end = nodes_df.at[idx, 'opening_hours'][day][0][0], nodes_df.at[idx, 'opening_hours'][day][-1][-1]
            if isinstance(start, str):  # Checking if start time is still a string
                start = datetime.strptime(start, "%H:%M").time()
            if isinstance(end, str):  # Checking if end time is still a string
                end = datetime.strptime(end, "%H:%M").time()
            start_dt = datetime.combine(datetime.today(), start)
            end_dt = datetime.combine(datetime.today(), end)
            appointment_start = start_dt + timedelta(minutes=random.randint(0, (end_dt - start_dt).seconds // 60 - 30))
            appointment_end = appointment_start + timedelta(minutes=30)
            nodes_df.at[idx, 'fixed_appointment'].append([day, appointment_start.time(), appointment_end.time()])
        return nodes_df
    
    current_date = datetime.now()
    last_visited = [current_date - timedelta(days=np.random.randint(1, max_last_visit)) for _ in range(num_nodes)]
    last_visited = [date.strftime("%Y-%m-%d") for date in last_visited]
    visiting_intervals = np.random.randint(visiting_interval_min, visiting_interval_max, size=num_nodes)
    durations = np.random.randint(30, 61, size=num_nodes)

    # Consolidate various attributes into a DataFrame for each node
    nodes_df = pd.DataFrame({
        "node_id": node_ids,
        "opening_hours": [generate_opening_hours(min_work_days) for _ in range(num_nodes)],
        "last_visited": last_visited,
        "Visiting Interval (days)": visiting_intervals,
        "on_site_time": durations
    })

    nodes_df.loc[nodes_df['node_id'] == 0, 'on_site_time'] = 0

    nodes_df.sort_values("node_id", inplace=True)
    nodes_df['days_since_last_visit'] = (current_date - pd.to_datetime(nodes_df['last_visited'])).dt.days
    nodes_df['priority'] = nodes_df['days_since_last_visit'] / nodes_df['Visiting Interval (days)']
    nodes_df['priority'] = nodes_df['priority'].apply(lambda x: 1 if x > 1 else x)
    nodes_df['fixed_appointment'] =  [[] for _ in range(num_nodes)]
    nodes_df['opening_hours'] = nodes_df['opening_hours']
    nodes_df = add_fixed_appointments(nodes_df, frac_fixed_app)

    # Generate random coordinates for each node
    coordinates = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(num_nodes)]
    coordinates_df = pd.DataFrame(coordinates, columns=['x', 'y'])
    nodes_df = pd.concat([nodes_df, coordinates_df], axis=1)

    def calculate_angles(nodes_df, home_x, home_y):
        """
        Calculate angles between the home node and other nodes.
        This helps in sorting nodes into angular clusters.
        """
        return np.degrees(np.arctan2(nodes_df['y'] - home_y, nodes_df['x'] - home_x)) % 360

    home_node = nodes_df.loc[home_node_id]
    home_x, home_y = home_node['x'], home_node['y']

    # Calculate the angles of all nodes with respect to the home node
    angles = calculate_angles(nodes_df, home_x, home_y)
    nodes_df['angle_to_home'] = angles

    # Create a distance matrix based on Euclidean distance
    def calculate_euclidean_distance_matrix(coords):
        return distance.cdist(coords, coords, 'euclidean')

    distance_matrix = calculate_euclidean_distance_matrix(nodes_df[['x', 'y']].values)
    distance_df = pd.DataFrame(np.round(distance_matrix, 2), columns=range(0, num_nodes), index=range(0, num_nodes))

    # Ensure the diagonal is zero to represent self-distance
    np.fill_diagonal(distance_df.values, 0)

    # add "dist_to_home" column
    nodes_df['dist_to_home'] = distance_df[home_node_id]
    return nodes_df, distance_df

In [348]:
nodes_df, time_matrix = create_nodes_dataframe(num_nodes=num_nodes, min_work_days=min_work_days, home_node_id=0, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=20, frac_fixed_app=percentage_of_appointments)
# create row copies for each fixed appointment
nodes_df = nodes_df.explode('fixed_appointment').reset_index(drop=True)
nodes_df['weekday_fixed_appointment'] = nodes_df['fixed_appointment'].apply(lambda x: x[0] if isinstance(x, list) else None)

- check eventualities of no fixed appointments or closed days

In [349]:
all_days = set(range(1, 8))
# Calculate open days as sets from the dictionary keys
nodes_df['open_days'] = nodes_df['opening_hours'].apply(lambda x: set(x.keys()))

# Calculate closed days
nodes_df['closed_days'] = nodes_df['open_days'].apply(lambda x: set(all_days - x))

# Filter for VIP nodes based on priority
VIP_nodes = nodes_df[nodes_df['priority'] > 0.8]

# For each VIP node, identify specific closed days that match the days off
if len(days_off) == 0:
    print("No days off specified.")

else:
    for index, row in VIP_nodes.iterrows():
        closed_days_off = set(row['closed_days']) & days_off
        if closed_days_off:
            closed_days_off_str = ', '.join(map(str, sorted(closed_days_off)))
            print(f"Warning: VIP node {row['node_id']} is closed on day(s) {closed_days_off_str}.")

- integrate logic for half days

In [350]:
# Generate valid lists as provided
def find_lists(current_list, current_sum, max_length, target_sum):
    if current_sum > target_sum or len(current_list) > max_length:
        return []
    if current_sum == target_sum and len(current_list) <= max_length:
        return [current_list]
    results = []
    for i in range(0, 8):
        adjusted_sum = current_sum + (i if i > 0 else 1)  # Adjust sum for 0s treated as 1
        results.extend(find_lists(current_list + [i], adjusted_sum, max_length, target_sum))
    return results

# Generate valid lists
valid_lists = find_lists([], 0, 7, 7)

# Function to repeat values in the list according to their integer value
def repeat_values(lst, overnight_trips=0):
    repeated_list = []
    for num in lst:
        if overnight_trips == 1:
            repeated_list.extend([num] * (num - 1) + [0] if num > 0 else [0])
        else:
            repeated_list.extend([num] * num if num > 0 else [0])
    return repeated_list

# Calculate days off for each list
def calculate_days_off(lst):
    repeated_lst = repeat_values(lst)
    days_off = []
    for i in range(min(len(repeated_lst), 7)):
        if repeated_lst[i] == 0:
            days_off.append(i + 1)  # Use 1-based indexing for days
    return days_off

# Calculate trip days for each list
def calculate_overnight_trips(lst):
    repeated_lst = repeat_values(lst, overnight_trips=1)  
    trip_days = []
    for i in range(min(len(repeated_lst), 7)):
        if repeated_lst[i] > 1:
            trip_days.append(i + 1)  # Use 1-based indexing for days
    return trip_days

def calculate_overnight_stays(lst):
    overnight_stays = 0
    for i in range(min(len(lst), 7)):
        if lst[i] > 1:
            overnight_stays += lst[i] - 1
    return overnight_stays

# Create a DataFrame
data = {
    'gaps': valid_lists,
    # 'Sum': [sum(lst) for lst in valid_lists],  # Calculating sum normally, 0s count as 0
    # 'Length': [len(lst) for lst in valid_lists],
    'n_overnight_trips': [calculate_overnight_stays(lst) for lst in valid_lists],
    'overnight_days': [calculate_overnight_trips(lst) for lst in valid_lists],
    'off_days': [calculate_days_off(lst) for lst in valid_lists],
    'n_days_off': [lst.count(0) for lst in valid_lists], 
}

options_df = pd.DataFrame(data)

# Filter the DataFrame based on the constraints
options_df = options_df[options_df['off_days'].apply(lambda x: all(day in x for day in days_off))]
options_df = options_df[options_df['overnight_days'].apply(lambda x: not any(day in x for day in no_overnight_stays))]
options_df = options_df[options_df['n_overnight_trips'].apply(lambda x: x <= max_overnight_stays)]
options_df = options_df[options_df['n_days_off'].apply(lambda x: x <= max_days_off)]
# add a column to df containing a dictionary counting the number of times an integer >1 appears in the list
options_df['blocks'] = options_df['gaps'].apply(lambda x: Counter([item for item in x if item > 0]))

In [351]:
if len(options_df) == 0:
    raise ValueError("No valid options found.")

- do simple solution finding for the above combinations, find best solution give cost of overnight stay, improve best solution

In [352]:
# plot_all_nodes_with_angles(nodes_df)

- in rare cases the below will make trouble because there are two large gaps and a cluster is entirely contained within the second largest leading to size = nan
- reintegrate distance matrix and replace dist_metric based on max dist with basis on variance within distances normalized by (?) total variance among nodes AND/OR use angle range as part of metric

In [353]:
def calculate_metric(nodes_df, global_max_dist, node_ids, cluster_id, print_ind_metrics=False):
    if len(node_ids) > 0:
        filtered_nodes_df = nodes_df[nodes_df['node_id'].isin(node_ids)]

        num_nodes_metric = len(filtered_nodes_df) / len(nodes_df)
        
        priority_metric = filtered_nodes_df['priority'].nlargest(int(0.5 * len(filtered_nodes_df))).mean()
        priority_metric = priority_metric / float(cluster_id.split('_')[0])

        max_dist_to_root = filtered_nodes_df['dist_to_home'].max()
        dist_metric = max_dist_to_root / global_max_dist

        # prevent any metric from being nan
        if np.isnan(num_nodes_metric):
            # print(f'Problems with num_nodes_metric for {cluster_id}')
            num_nodes_metric = 0.5
        if np.isnan(priority_metric):
            # print(f'Problems with priority_metric for {cluster_id}')
            priority_metric = 0.3
        if np.isnan(dist_metric):
            # print(f'Problems with dist_metric for {cluster_id}')
            dist_metric = 0.5

        metric = num_nodes_metric + dist_metric / 6

        if print_ind_metrics:
            print(f'Cluster: {cluster_id}')
            print(f"Number of nodes metric: {num_nodes_metric}")
            print(f"Priority metric: {priority_metric}")
            print(f"Distance metric: {dist_metric}")
            print(f"Overall metric: {metric}")
    else:
        metric = 0

        if print_ind_metrics:
            print('Found a cluster wihtout nodes')
    
    return metric

def adjust_angles(clusters, nodes_df, angle_sizes, degree_adj, global_max_dist, cluster_sizes, total_span, verbose):
    metrics = {}
    for cluster_id, node_ids in clusters.items():
        metrics[cluster_id] = calculate_metric(nodes_df, global_max_dist, node_ids, cluster_id)

    total_metric = sum(metrics.values())
    # total_days = sum of each key multiplied by the value in cluster_sizes
    total_days = sum([key * value for key, value in cluster_sizes.items()])
    base_metric = total_metric / total_days
    
    target_metrics = {}
    for cluster, metric in metrics.items():
        size = float(cluster.split('_')[0])
        target_metrics[cluster] = base_metric * size

    new_angle_sizes = angle_sizes.copy()  # Copy existing angle sizes to modify
    
    deviations = {}
    for cluster_id, metric in metrics.items():
        size = float(cluster_id.split('_')[0])
        soll_metric = target_metrics[cluster_id]
        deviation = metric - soll_metric
        deviations[cluster_id] = deviation
        new_angle_sizes[cluster_id] -= deviation * degree_adj

    # Normalize the new angles to ensure they sum to total_span
    total_new_angles = sum(new_angle_sizes.values())
    scale_factor = total_span / total_new_angles
    for cluster_id in new_angle_sizes:
        new_angle_sizes[cluster_id] *= scale_factor

    if verbose:
        print("Deviations, metrics, and new angle sizes:")
        for cluster_id in clusters:
            print(f"Cluster {cluster_id} with deviation {round(deviations[cluster_id], 2)}, "
                  f'and initial angle size {round(angle_sizes[cluster_id], 2)}° '
                  f"has new angle size {round(new_angle_sizes[cluster_id], 2)}°.")

    return new_angle_sizes

def custom_clustering(nodes_df, cluster_sizes, precision, home_node_id=0, verbose=False, visual=False):
    # remove the home node from the nodes_df
    if nodes_df.index[0] == 0:
        nodes_df_copy = nodes_df.drop(0).copy()
    
    clusters = {}
    for size, count in cluster_sizes.items():
        for i in range(count):
            clusters[f'{size}_day_trip_{i}'] = []
    
    angles = sorted(nodes_df_copy['angle_to_home'])
    diffs = [angles[i + 1] - angles[i] for i in range(len(angles) - 1)]
    diffs.append(360 - angles[-1] + angles[0])
    
    max_gap = max(diffs)
    gap_start = angles[diffs.index(max_gap)]
    gap_end = angles[(diffs.index(max_gap) + 1) % len(angles)]

    max_gap = max(diffs)
    total_span = 360 - max_gap

    if verbose == True:
        print(f"Largest gap spans from {gap_start}° to {gap_end}°, covering {max_gap}° leaving a total span of {total_span} for locations.")

    total_equivalent_degrees = sum(count * size for size, count in cluster_sizes.items())
    base_degree = total_span / total_equivalent_degrees

    angle_sizes = {}
    for size, count in cluster_sizes.items():
        # Calculate the angular size for each cluster of this size
        cluster_angle_size = base_degree * size
        for i in range(count):
            cluster_id = f'{size}_day_trip_{i}'
            angle_sizes[cluster_id] = cluster_angle_size
    global_max_dist = nodes_df_copy['dist_to_home'].max()

    cluster_start = gap_end
    degree_adj = total_span / 7

    # Initial assignment of nodes to clusters
    for i in range(precision):
        current_angle = cluster_start  # Reset the start angle for each precision iteration
            
        # add the home node to each cluster
        for key in clusters.keys():
            clusters[key] = [home_node_id]

        # Assign nodes to clusters based on their angle to the home node
        for cluster_id, size in angle_sizes.items():
            start_angle = current_angle
            # round up to the nearest integer
            start_angle = int(start_angle)
            end_angle = (current_angle + size) % 360
            end_angle = int(np.ceil(end_angle))
            # Ensuring all nodes are assigned, handling the wrap-around scenario more cleanly
            if end_angle < start_angle:  # This handles the case where the segment wraps past 360 degrees
                nodes_in_cluster = [index for index, row in nodes_df_copy.iterrows() if 
                                    (row['angle_to_home'] >= start_angle or row['angle_to_home'] < end_angle)]
            else:  # No wrap-around, normal case
                nodes_in_cluster = [index for index, row in nodes_df_copy.iterrows() if 
                                    (start_angle <= row['angle_to_home'] < end_angle)]

            clusters[cluster_id] = [home_node_id] + nodes_in_cluster
            current_angle = end_angle

        if (i == 0 and verbose):
            print("Initial clusters:")
            for key, value in clusters.items():
                print(f"{key}: {value}")

        if i % 10 == 0 and visual:
            plot_refined_clusters(clusters, nodes_df)
        
        angle_sizes = adjust_angles(clusters, nodes_df_copy, angle_sizes, degree_adj, global_max_dist, cluster_sizes, total_span, verbose)
        degree_adj *= 0.95

        sum_of_angles = sum(angle_sizes.values())
        if verbose:
            print(f"Sum of angles: {sum_of_angles} vs. total span: {total_span}")
        
    return clusters

def fit_blocks(blocks, gaps, solution, solutions, index=0):
    if index == len(gaps):  # Check if we've addressed all gaps
        if not blocks:  # Ensure all blocks have been used
            solutions.append(solution)
        return
    
    # Attempt to fit each block into the current gap
    for i, block in enumerate(blocks):
        # ['1_day_trip_5', '1_day_trip_0', '1_day_trip_1', '1_day_trip_3', '1_day_trip_4', '1_day_trip_2']
        block_size = int(block[0])  # Extract size from block identifier, e.g., "2A" -> 2
        if block_size <= gaps[index]:  # Check if block can fit in the current gap
            # Setup for recursion: remove the block and reduce the gap size
            new_blocks = blocks[:i] + blocks[i+1:]  # Remove current block
            new_solution = [lst[:] for lst in solution]  # Copy solution to modify
            new_solution[index].append(block)  # Add block to current gap's solution
            
            # Reduce the gap by the size of the block
            new_gaps = gaps[:]
            new_gaps[index] -= block_size
            
            # Move to next gap if current gap is exactly filled, otherwise continue
            if new_gaps[index] == 0:
                fit_blocks(new_blocks, new_gaps, new_solution, solutions, index + 1)
            else:
                fit_blocks(new_blocks, new_gaps, new_solution, solutions, index)

def count_fixed_appointments(weekly_schedule, nodes_df):
    count = 0
    for i, cluster_day in enumerate(weekly_schedule):
        start_day = i + 1
        if 'day_off' not in cluster_day:
            duration = int(cluster_day.split('_')[0])
            end_day = start_day + duration
            for day in range(start_day, end_day + 1):
                count += nodes_df[(nodes_df['cluster'] == cluster_day) & (nodes_df['weekdays_fixed_appointments'] == day)].shape[0]
    return count

In [359]:
visual = False
verbose = True
options_df = options_df.sort_values(by='n_overnight_trips')
print('there are ', len(options_df), 'options that will be tested')
for index, row in options_df.iterrows():
    blocks = row['blocks']
    gaps = row['gaps']
    clusters = custom_clustering(nodes_df, blocks, precision=50, verbose=False, visual=False)

    if verbose:
        for cluster, nodes in clusters.items():
            print(f"Cluster {cluster}:")
            print(f"Average node priority: {round(nodes_df.loc[nodes_df['node_id'].isin(nodes), 'priority'].mean(), 2)}")
            print(f"Average distance to home: {round(nodes_df.loc[nodes_df['node_id'].isin(nodes), 'dist_to_home'].mean(), 2)}")
            print(f"Count of nodes: {len(nodes)}")

    if visual:
        plot_refined_clusters(clusters, nodes_df)

    node_to_cluster = {node: cluster for cluster, nodes in clusters.items() for node in nodes}
    nodes_df['cluster'] = nodes_df['node_id'].map(node_to_cluster)

    # store positions of 0s in gaps to remove and add them to the solution after fitting blocks
    zero_positions = [i for i, gap in enumerate(gaps) if gap == 0]
    gaps = [gap for gap in gaps if gap > 0]
    
    blocks = list(nodes_df['cluster'].unique())
    solution = [[] for _ in gaps]  # Initialize solution structure for each gap
    solutions = []
    if verbose:
        print(f'Blocks: {blocks}, Gaps: {gaps}')
    fit_blocks(blocks, gaps, solution, solutions)

    # Add 0s back to the solution as 'day-off' clusters
    for solution in solutions:
        for position in zero_positions:
            solution.insert(position, ['1_day_off'])
    solutions = [[item for sublist in outer_list for item in sublist] for outer_list in solutions]
    repeated_list = []
    for sublist in solutions:
        new_sublist = []
        for item in sublist:
            count = int(item.split('_')[0])  # Extract the number of repetitions
            new_sublist.extend([item] * count)  # Repeat the item
        
        repeated_list.append(new_sublist)
    
    if verbose:
        print(f'Found {len(repeated_list)} solutions: {repeated_list}')

    best_count = -float('inf')
    best_list = None
    # for each list capturing the allocations of clusters to weekdays
    for i, sublist in enumerate(repeated_list):
        if verbose:
            print(f'solution {i+1}/{len(repeated_list)}')
        count = 0
        # for each cluster in the list
        for cluster in set(sublist):
            if not 'day_off' in cluster:
                # Access the indices of the cluster in the week
                relevant_weekdays = [i + 1 for i, gap in enumerate(sublist) if gap == cluster]
                
                
                # get a list of non-unique fixed appointments
                fixed_appointment_day = list(nodes_df[nodes_df['cluster'] == cluster]['weekday_fixed_appointment'])
                
                cluster_count_fixed = len([entry for entry in fixed_appointment_day if entry in relevant_weekdays])
                initial_count = count
                count += cluster_count_fixed

                closed_days = list(nodes_df[nodes_df['cluster'] == cluster]['closed_days'].dropna())
                flat_closed_days = []
                for s in closed_days:
                    flat_closed_days.extend(s)
                
                cluster_count_closed = len([entry for entry in flat_closed_days if entry in relevant_weekdays])
                count -= cluster_count_closed
                if verbose:
                    print(f'Relevant weekdays: {relevant_weekdays}')
                    print(f'Fixed appointment days: {fixed_appointment_day}')
                    print("Closed on:", flat_closed_days)
                    print(f'added {cluster_count_fixed} and deducted {cluster_count_closed} from {initial_count}')

        if count > best_count:
            if verbose:
                print(f'replacing {best_count} with {count}')
            best_count = count
            best_list = sublist

    if verbose:
        print(f'Best solution had a count of {best_count}')

    mapping_dict = {}
    for i, cluster in enumerate(best_list):
        indices = [j+1 for j, x in enumerate(best_list) if x == cluster]
        mapping_dict[cluster] = set(indices)

    nodes_df['cluster'] = nodes_df['cluster'].map(mapping_dict)

    clusters = nodes_df['cluster'].drop_duplicates().tolist()
    # assign "to be visited on a day off" nodes to the cluster
    closed_day_problems = nodes_df[nodes_df.apply(lambda row: row['cluster'].issubset(row['closed_days']), axis=1)][['node_id', 'closed_days', 'x', 'y']]
    closed_day_problems = closed_day_problems[closed_day_problems['node_id'] != 0]
    if verbose:
            print(f'number of nodes to be reassigned: {len(closed_day_problems)}')
    for sub_row in closed_day_problems.iterrows():
        closed_on = sub_row[1]['closed_days']
        # find possible clusters (i.e.: not all closed on days contained)
        possible_clusters = [cluster for cluster in clusters if not cluster.issubset(closed_on)]
        possible_nodes = nodes_df[nodes_df['cluster'].isin(possible_clusters)]['node_id']
        possible_nodes = possible_nodes[possible_nodes != 0]
        
        if verbose:
            print(f'closed_on: {closed_on}, possible_clusters: {possible_clusters}')
        # remove 0 from row and col of time_matrix
        time_matrix_sub = time_matrix.drop(0, axis=0).drop(0, axis=1)
        closest_node = time_matrix_sub.loc[sub_row[1]['node_id'], possible_nodes].idxmin()
        closest_node_cluster = nodes_df.loc[closest_node, 'cluster']
        if verbose:
            print(f'reassigning node {sub_row[1]["node_id"]} with coordinates ({sub_row[1]["x"]}, {sub_row[1]["y"]}) to cluster {closest_node_cluster} since node {closest_node} with coordinates ({nodes_df.loc[closest_node, "x"]}, {nodes_df.loc[closest_node, "y"]}) is the closest node in a possible cluster')
        # update the cluster of the node with the set defining the new cluster preventing "Must have equal len keys and value when setting with an iterable"
        nodes_df.at[sub_row[0], 'cluster'] = closest_node_cluster
    
    # assign nodes with fixed appointments always to the cluster that contains the fixed appointments visit day in cluster index
    for index, row in nodes_df.iterrows():
        appointment = row['weekday_fixed_appointment']
        if (appointment not in row['cluster']) and not pd.isnull(appointment):
            if verbose:
                print(f'{row["node_id"]} has a fixed appointment on {appointment} but is assigned to cluster {row["cluster"]}')
            # find the cluster that contains the appointment day
            cluster_with_appointment = [cluster for cluster in clusters if appointment in cluster]
            if verbose:
                print(f'Assigning node {row["node_id"]} to cluster {cluster_with_appointment}')
            nodes_df.at[index, 'cluster'] = cluster_with_appointment[0]

there are  4 options that will be tested
Cluster 1_day_trip_0:
Average node priority: 0.44
Average distance to home: 41.28
Count of nodes: 10
Cluster 1_day_trip_1:
Average node priority: 0.56
Average distance to home: 71.67
Count of nodes: 16
Cluster 1_day_trip_2:
Average node priority: 0.77
Average distance to home: 50.72
Count of nodes: 14
Cluster 1_day_trip_3:
Average node priority: 0.54
Average distance to home: 59.28
Count of nodes: 14
Cluster 1_day_trip_4:
Average node priority: 0.49
Average distance to home: 56.85
Count of nodes: 13
Blocks: ['1_day_trip_4', '1_day_trip_2', '1_day_trip_3', '1_day_trip_1', '1_day_trip_0'], Gaps: [1, 1, 1, 1, 1]
Found 120 solutions: [['1_day_trip_4', '1_day_trip_2', '1_day_trip_3', '1_day_trip_1', '1_day_off', '1_day_trip_0', '1_day_off'], ['1_day_trip_4', '1_day_trip_2', '1_day_trip_3', '1_day_trip_0', '1_day_off', '1_day_trip_1', '1_day_off'], ['1_day_trip_4', '1_day_trip_2', '1_day_trip_1', '1_day_trip_3', '1_day_off', '1_day_trip_0', '1_day_off

- Integrate lunch breaks for locations
- deduct on_site_time from closing times
- double check that in all cases opening hours are kept for all relevant days

In [355]:
def time_to_minutes(t):
    return t.hour * 60 + t.minute + t.second

def adjust_opening_hours(row):
    clusters = row['cluster']
    first_day = min(clusters)
    opening_hours = row['opening_hours']
    fixed_appointment = row['fixed_appointment']
    adjusted_hours = {}
    if isinstance(fixed_appointment, list):
        day, app_start, app_end = fixed_appointment[0], fixed_appointment[1], fixed_appointment[2]
        adjusted_open = time_to_minutes(app_start) + 1440 * (day - first_day) - margin 
        adjusted_close = time_to_minutes(app_end) + 1440 * (day - first_day) + margin
        adjusted_hours[day] = [adjusted_open, adjusted_close]
    else:
        for day, intervals in opening_hours.items():
            if day in clusters:  # Ensure we only adjust days that are in clusters
                adjusted_intervals = []
                for start, end in intervals:
                    # Adjust each interval's start and end times
                    adjusted_start = time_to_minutes(start) + 1440 * (day - first_day)
                    adjusted_end = time_to_minutes(end) + 1440 * (day - first_day)
                    adjusted_intervals.append((adjusted_start, adjusted_end))
                adjusted_hours[day] = adjusted_intervals                
    return adjusted_hours

nodes_df['adjusted_opening_hours'] = nodes_df.apply(adjust_opening_hours, axis=1)

In [356]:
# bring sets in the cluster column into a format that allows grouping
nodes_df['cluster'] = nodes_df['cluster'].apply(lambda x: frozenset(x))
refined_clusters = nodes_df.groupby('cluster')['node_id'].apply(list).to_dict()

In [357]:
depot_node_data = nodes_df[nodes_df['node_id'] == 0].iloc[0]  # Assuming there is always a row for node 0 in the original DataFrame

result_dfs = {}
for index, group in nodes_df.groupby('cluster'):
    # Check if depot node is in the current group
    if 0 not in group['node_id'].values:
        # Append depot node data to the group
        group = pd.concat([pd.DataFrame([depot_node_data]), group], ignore_index=True)
    # Now group is guaranteed to include the depot node
    result_dfs[index] = group[['node_id', 'priority', 'adjusted_opening_hours', 'cluster', 'on_site_time']]

In [361]:
# replace columns adjusted_opening_hours and fixed_appointment in nodes_df with strings and save to excel
nodes_df['adjusted_opening_hours'] = nodes_df['adjusted_opening_hours'].apply(str)
nodes_df['fixed_appointment'] = nodes_df['fixed_appointment'].apply(str)
nodes_df.to_csv('nodes_df.csv', index=False)

# Find route

- nodes with fixed appointments should have prio 1
- adapt for trips > 2
- correct index for printing dropped nodes
- optimize route beyond priority choice

In [358]:
sub_nodes_df = result_dfs[list(result_dfs.keys())[0]]
solutions = {}
route_lists = {}
route_lists_with_travel = {}

def minutes_to_hhmm(minutes_am):
    minutes_am = minutes_am % 1440  # Ensure minutes are within a day
    minutes = minutes_am % 60
    hours = (minutes_am - minutes) // 60  # Use integer division for hours
    return f'{hours:02}:{minutes:02}'

def create_data_model(sub_nodes_df, sub_time_matrix):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = sub_time_matrix
    data['windows'] = sub_nodes_df['adjusted_opening_hours'].tolist()
    data['priorities'] = sub_nodes_df['priority'].tolist()
    data['num_vehicles'] = 1
    data['on_site_time'] = sub_nodes_df['on_site_time'].tolist()
    data['depot'] = 0
    return data

def return_route_and_times(solution, manager, routing, original_node_ids, data):
    """Returns the route along with the start times at each node."""
    index = routing.Start(0)  # Start at the depot.
    route_with_travel = []
    route_without_travel = []
    time_dimension = routing.GetDimensionOrDie('total_time')  # Make sure this matches the dimension name used

    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        original_node_id = original_node_ids[node_index]  # Map back to original node ID
        time_var = time_dimension.CumulVar(index)
        start_time = solution.Min(time_var)
        end_time = start_time + data['on_site_time'][node_index]  # Include on-site time
        route_with_travel.append((original_node_id, start_time, end_time))  # Include end time for better clarity
        route_without_travel.append((original_node_id, start_time))  # Include end time for better clarity
        next_index = solution.Value(routing.NextVar(index))
        
        travel_time = routing.GetArcCostForVehicle(index, next_index, 0) - data['on_site_time'][index]  # Get travel time
        route_with_travel.append(("road", travel_time))
        
        index = next_index

    # Add the final node
    final_node_index = manager.IndexToNode(index)
    final_node_id = original_node_ids[final_node_index]
    final_time_var = time_dimension.CumulVar(index)
    final_start_time = solution.Min(final_time_var)
    final_end_time = final_start_time + data['on_site_time'][final_node_index]
    route_with_travel.append((final_node_id, final_start_time, final_end_time))
    route_without_travel.append((final_node_id, final_start_time))

    return route_with_travel, route_without_travel

def print_route(route_with_times):
    """Prints the route in the desired format."""
    route_str = ""
    for segment in route_with_times:
        if segment[0] == "road":
            route_str += f" - road ({segment[1]}) - "
        else:
            node_id, start_time, end_time = segment
            route_str += f"{node_id} ({minutes_to_hhmm(start_time)}-{minutes_to_hhmm(end_time)})"
    print(route_str)

# def solve_vrp(key, sub_nodes_df):
if True:
    max_travel_time = 10000 if len(sub_nodes_df['cluster'].iloc[0]) == 1 else 20000 # adjust
    nodes = sub_nodes_df['node_id'].tolist()
    sub_time_matrix = time_matrix.loc[nodes, nodes].values.tolist()
    sub_time_matrix = [[int(x) for x in row] for row in sub_time_matrix]
    data = create_data_model(sub_nodes_df, sub_time_matrix)
    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node] + data['on_site_time'][from_node]
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    routing.AddDimension(
        transit_callback_index,
        slack,  # upper bound for slack / waiting time
        max_travel_time,  # upper bound for vehicle maximum travel time
        False,  # start cumul to zero
        "total_time"
    )
    time_dimension = routing.GetDimensionOrDie("total_time")
    time_dimension.SetGlobalSpanCostCoefficient(span_cost_coefficient)

    # PENALTY
    for location_index, priority in enumerate(data['priorities']):
        index = manager.NodeToIndex(location_index)
        if index == 0:
            continue
        else:
            routing.AddDisjunction([index], int(round((priority*100)**2*penalty_factor, 0)))

    # OPENING HOURS, LUNCH AND OVERNIGHT BREAKS
    for location_index, windows in enumerate(data['windows']):
        index = manager.NodeToIndex(location_index)
        days = len(windows)
        if days > 1:
            if index < manager.GetNumberOfNodes():
                work_start = working_hours[0]
                work_end = working_hours[1] + 1440
                latest_start = max(work_start, windows[0][0])
                earliest_end = min(work_end, windows[-1][1])
                # print(f'setting the time window from {latest_start} to {earliest_end} for node {location_index}')
                time_dimension.CumulVar(index).SetRange(latest_start, earliest_end)
            for day in range(1, days):
                # print(f'and removing time between days from {working_hours[1] + 1440 * (day-1)} to {working_hours[0] + 1440 * day} for node {location_index}')
                time_dimension.CumulVar(index).RemoveInterval(windows[day-1][1], windows[day][0])
                time_dimension.CumulVar(index).RemoveInterval(working_hours[1] + 1440 * (day-1), working_hours[0] + 1440 * day)
        else:
            if windows[0][0] > 1440:
                # print('work on day 2')
                work_start = working_hours[0] + 1440
                work_end = working_hours[1] + 1440
                # print(f'work starts at {work_start} and ends at {work_end}')
                # print(f'window starts at {windows[0][0]} and ends at {windows[0][1]}')
                day_start = max(windows[0][0], work_start)
                day_end = min(windows[0][1], work_end)
            else:
                day_start = windows[0][0]
                day_end = windows[0][1]
            # print(f'setting the time window from {day_start} to {day_end} for node {location_index}')
            time_dimension.CumulVar(index).SetRange(day_start, day_end)  

    node_visit_transit = {}
    for index in range(routing.Size()):
        node = manager.IndexToNode(index)
        node_visit_transit[index] = data['on_site_time'][node]

    lunch_break_interval = routing.solver().FixedDurationIntervalVar(
        lunch_start, lunch_end, lunch_duration, False, 'lunch_break'
    )

    # Assign the break interval to the single vehicle
    time_dimension.SetBreakIntervalsOfVehicle([lunch_break_interval], 0, node_visit_transit)

    # Instantiate route start and end times to produce feasible times
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(0)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(0)))

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.local_search_metaheuristic = (
    #     routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    search_parameters.time_limit.seconds = 300
    search_parameters.log_search = False

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

#     if solution:
#         dropped = []
#         for node in range(routing.Size()):
#             if routing.IsStart(node) or routing.IsEnd(node):
#                 continue
#             if solution.Value(routing.NextVar(node)) == node:
#                 dropped.append(manager.IndexToNode(node))
#         if len(dropped) > 0:
#             print(f"dropped from {key}: {dropped}")
#         return key, return_route_and_times(solution, manager, routing, nodes, data)
        
#     else:
#         print(f"No solution for key {key}")
#         return key, None
    
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     future_to_key = {executor.submit(solve_vrp, key, sub_nodes_df): key for key, sub_nodes_df in result_dfs.items()}
#     for future in concurrent.futures.as_completed(future_to_key):
#         key = future_to_key[future]
#         # print(f"Finding solution for key: {key}")
#         try:
#             key, result = future.result()
#             if result:
#                 route_lists_with_travel[key] = result[0]
#                 route_lists[key] = result[1]
#         except Exception as e:
#             print(f"Error for key {key}: {e}")

# for key, route in route_lists_with_travel.items():
#     print(f"Route for key {key}:")
#     print_route(route)

KeyError: 0

# check and visualize results

In [ ]:
route_and_times = route_lists[list(route_lists.keys())[0]]
route_df = pd.DataFrame(route_and_times, columns=['node_id', 'arrival_time'])
merged_df = pd.merge(nodes_df, route_df, on='node_id', how='left')[['adjusted_opening_hours', 'arrival_time', 'node_id']]
merged_df = merged_df[merged_df['node_id'] != 0]

def check_within_ranges(arrival_time, ranges):
    if len(ranges) == 1:
        return ranges[0][0] <= arrival_time <= ranges[0][1]
    elif len(ranges) == 2:
        return (ranges[0][0] <= arrival_time <= ranges[0][1]) or (ranges[1][0] <= arrival_time <= ranges[1][1])
    return False

merged_df['time_check'] = merged_df.apply(
    lambda row: check_within_ranges(row['arrival_time'], row['adjusted_opening_hours']), axis=1
)

if merged_df['time_check'].all():
    print('VIOLATION OF TIME CONSTRAINTS')
    print(merged_df[merged_df['time_check'] == False])

# merged_df[['node_id', 'adjusted_opening_hours', 'arrival_time', 'time_check']]

- print more info (opening hours, fixed appointments, priority, decisions [removing a node, replacing a node to another day, priority, etc])
- use data to fine tune hyperparameters

evtl:
- Store state and if it was possible to find routes for all solutions iteratively add nodes based on node priority and distance to root node (those further away should be included more likely)
- Test Discrete Priority (must be visited this week; must be visited next week; ...)
- Compare routes with and without overnight stays / large clusters

In [ ]:
plot_refined_clusters(refined_clusters, nodes_df, home_node_id=0)

In [ ]:
plot_all_cluster_routes(route_lists, nodes_df)